In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
import heapq
import os
from decimal import Decimal
np.set_printoptions(threshold=np.nan) 
home_path = os.getcwd().replace('BC1/notebooks','')
import cPickle as pickle
import pandas as pd
from scipy.signal import find_peaks
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.cluster import SpectralClustering
data_region = 'CS10706F'
                                                                                                            

In [2]:
def find_tubes(signal, percentage, number_tubes):
    """
    Function to find the first and last indexes of each
    tube inside the cell. In other words, this function
    get the signal when the cesium capsule is inside
    the cell.
    Parameters:
    ----------
    signal : array
        Array with signal profile from Cesium Scan.
    percentage : float
        Constant number of the percentage to cut the
        ADC Counts axis of the signal, in order to
        find the tubes in the cell.
    number_tubes : int
        Expected number of tubes inside the cell.
    Returns:
    -------
    indexes : list
        List with tuples containing the first and
        last indexes of each tube found.
    """
    # get peak indexes
    height = np.max(signal) * percentage
    peak_idx, _ = find_peaks(signal, height=height)

    # distance between peaks
    peak_dist = np.diff(peak_idx)

    # positional argument
    pos = np.array([i+1 for i, dist in enumerate(peak_dist) if dist >= 100])
    pos = np.insert(pos, 0, 0)
    pos = np.insert(pos, len(pos), len(peak_dist)+1)

    # loop througout all the tubes
    indexes = []
    for tube in range(number_tubes):
        # get the peaks indexes for each tube
        peaks_per_tube = peak_idx[pos[tube]:pos[tube+1]]

        # get mean distance between peaks/periods
        mean_period = np.mean(np.diff(peaks_per_tube))

        # get first and last indexes of this tube
        first = int(peaks_per_tube[0] - (6*mean_period))
        last = int(peaks_per_tube[-1] + (6*mean_period))
        indexes.append((first, last))

    return indexes


In [3]:
    def create_image(data_per_idx,map_file):
        """
        Private method that maps each channel into a pixel in the
        MA-PMT grid and creates its image.

        Args:
        ----
        data_per_idx (array): one realization (trigger/time unit) per channel
        map_file (str): complete path to the mapping file

        Returns:
        -------
        data_matrix (2D array): image of the MA-PMT
        """
        # get mapping for image pixels
        pixel_map = np.loadtxt(map_file)

        data_matrix = np.zeros([8, 8])

        for i in range(8):
            for j in range(8):
                if pixel_map[i, j] == 0:
                    continue
                else:
                    channel = int(pixel_map[i, j]-1)
                    data_matrix[i, j] = data_per_idx[channel]

        return data_matrix

In [4]:
#Serpentine mode vectoring
#transform into a vector an array received from Image Channel
def vector_serpentina():
    """
        Private method that returns the transformation
        of an matrix in array using the serpentine method

        Args:
        ----
        None 

        Returns:
        -------
        lista (1D array): vectorization result
    """
    
    lista=np.array([])
    #Receive array of "create_image" function from file Image Channel
    vetor=ma_pmt_image
    for i in range (8):

        if(i==1 or i==3 or i==5 or i==7):
            #Reverse the order of elements in an array along the given axis.
            vetor=np.flip(ma_pmt_image, 1) 
        else:
            #Does not require even lines to flip
            vetor=ma_pmt_image

        for j in range(8):
            lista=np.append(lista,vetor[i,j])
    return lista

In [5]:
#Desvectorize serpentine mode
def desvector_serpentina(vetor):
    """
        Private method that returns the transformation
        of an array in matrix using the serpentine method

        Args:
        ----
        Vetor(1D Array):Vector that we will transform into matrix

        Returns:
        -------
        v (2D array): transformation result
    """
    #Using the copy, to avoid the error of change in the serpentine matrix
    teste=vetor.copy()
    #Gives a new shape to an array without changing its data.
    v = teste.reshape((8, 8))
    for i in range (8):

        if(i==1 or i==3 or i==5 or i==7):
            bloco = v[i,:8]
            #Reverse the order of elements in an array along the given axis.
            bloco[:] = np.flip(bloco.copy(),0)
        
    return v

In [6]:
#opening the file
file = open(home_path+'/processed_data/Dados BC1/BC1.CS10706F.LBA66.FOCON.pkl','rb')
cesium_scan = pickle.load(file)
single_anode = 2
single_data = cesium_scan['data'][:, single_anode]

#Path to the channel reading mapping file
map_path = home_path+'/processed_data/Dados BC1/mapping.txt'

In [7]:
# Find the tubes inside the cell using the
# peaks information of the single anode PMT
tubes_idx = find_tubes(single_data,percentage=0.6,number_tubes=6)

In [8]:
multi_data = cesium_scan['data']


In [9]:
separate_peaks = []
for k in range(len(tubes_idx)):
    separate_peaks.append(multi_data[tubes_idx[k][0]:tubes_idx[k][1]])

In [10]:
#Uses the vector_serpentina method to vectorize all data array entries
#It returns a 165X64 matrix, with the values of the Data_Array.npy files mapped
for l in range(separate_peaks[0].shape[0]):
    #ma_pmt_image receives array from imported file after passing through "create image"
    ma_pmt_image = create_image(separate_peaks[0][l,:], map_path)
    if (l==0):
        #Used to initialize matriz_serpentina to avoid error in vstack
        matriz_serpentina=vector_serpentina()
    else:
        #Stack arrays in sequence vertically (row wise).
        matriz_serpentina=np.vstack((  matriz_serpentina, vector_serpentina()  ))

In [11]:
#Comparing if the matrices are equal
np.array_equal(ma_pmt_image,desvector_serpentina(matriz_serpentina[169]))

True

In [12]:
pixel_map = np.loadtxt(map_path)
print pixel_map

[[  0.   0.   0.  48.  46.   0.   0.   0.]
 [  0.  44.  42.  40.  38.  36.  34.   0.]
 [ 19.  32.  30.  28.  26.  23.  25.   0.]
 [ 27.  29.  31.  33.  35.  37.  39.  41.]
 [ 43.  45.  47.  24.  22.  20.  18.  16.]
 [  0.  14.  12.   8.   6.   5.  10.  21.]
 [  0.   1.   2.   9.  17.   7.  11.   0.]
 [  0.   0.   0.  13.  15.   0.   0.   0.]]
